In [22]:
import gc
import cv2
import dlib
from PIL import Image
import imutils
import numpy as np
from sklearn.decomposition import PCA
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

In [23]:
# Displayinh the number of GPUs available
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)

Num GPUs Available:  1


In [24]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [25]:
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

In [26]:
# load a test image, resize it, and convert it to grayscale
def load_image(path, show=False, gray_show=False):
    test_image = cv2.imread(path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

    test_image = imutils.resize(test_image, width=500)
    gray = cv2.cvtColor(test_image, cv2.COLOR_RGB2GRAY)

    if show:
        plt.imshow(test_image)
        plt.show()
        if gray_show:
            plt.imshow(gray, cmap='gray')
            plt.show()
    return test_image, gray

def detect_faces(face_detector, gray, img, scale_factor, frame_num, show_multi_faces = False):
    detections = haar_cascade.detectMultiScale(gray, scaleFactor=scale_factor, minNeighbors=5, minSize=(150, 150), flags=cv2.CASCADE_SCALE_IMAGE)
    #Checking if there are more than 1 detected faces
    if len(detections) > 1:
        print('multiple faces', frame_num)
        if detections[0][2] > detections[1][2]:
            face = detections[0]
            other = detections[1]
        else:
            face = detections[1]
            other = detections[0]
        if show_multi_faces:
            print('largest and chosen face')
            fX, fY, fW, fH = face[0],face[1], face[2], face[3]
            plt.imshow(gray[fY:fY + fH, fX:fX + fW], cmap='gray')
            plt.show()
            print('not chosen face')
            fX, fY, fW, fH = other[0], other[1], other[2], other[3]
            plt.imshow(gray[fY:fY + fH, fX:fX + fW], cmap='gray')
            plt.show()
    elif len(detections) == 1:
        face = detections[0]
    # if no face detected
    elif len(detections) == 0:
        print('No face detected', frame_num)
        return False
    fX, fY, fW, fH = face[0], face[1], face[2], face[3]
    face_img = Image.fromarray(img[fY:fY + fH, fX:fX + fW])
    face_img = face_img.resize((224, 224))
    #face_img.show()
    return face_img

In [27]:
# haar Cascade path
cascade_path = '../models/haarcascade_frontalface_default.xml'
# create haar cascade
haar_cascade = cv2.CascadeClassifier(cascade_path)
face_detector = 'haar'

frame_start = 63000
frame_end = 220000
frames = range(frame_start, frame_end + 1)

scale_factor = 1.03

folder_path = '../frames/P11410/sleepy/frame'

In [28]:
def get_embeddings(frames, folder_path, face_detector, model, scale_factor, show=False, show_multi_faces=False):
    no_face_count = 0
    embeddings = []
    for frame in frames:
        if frame % 1000 == 0:
            print(frame)
            gc.collect()
        path = folder_path + str(frame) + '.jpg'
        test_img, gray = load_image(path, show=False)
        face = detect_faces(face_detector, gray, test_img, scale_factor, frame, show_multi_faces=False)

        if not face:
            print("No face detected", frame)
            no_face_count += 1
            continue

        face_array = np.asarray(face)
        pixels = face_array.astype('float32')
        samples = np.expand_dims(pixels, axis=0)
        samples = preprocess_input(samples, version=2)
        embedding = model.predict(samples, verbose=0)
        embeddings.append(embedding[0])
    print(no_face_count)
    return embeddings

embeddings = get_embeddings(frames, folder_path, face_detector, model, scale_factor)
np.save('embeddings_sub10_sleepy.npy', embeddings)

63000
64000
multiple faces 64623
65000
66000
67000
multiple faces 67266
multiple faces 67267
multiple faces 67270
multiple faces 67274
multiple faces 67403
multiple faces 67404
multiple faces 67405
multiple faces 67406
multiple faces 67407
multiple faces 67408
multiple faces 67409
multiple faces 67410
multiple faces 67411
multiple faces 67412
multiple faces 67413
68000
69000
70000
71000
No face detected 71128
No face detected 71128
No face detected 71130
No face detected 71130
No face detected 71131
No face detected 71131
No face detected 71132
No face detected 71132
No face detected 71133
No face detected 71133
No face detected 71134
No face detected 71134
No face detected 71135
No face detected 71135
No face detected 71136
No face detected 71136
No face detected 71137
No face detected 71137
No face detected 71138
No face detected 71138
No face detected 71139
No face detected 71139
No face detected 71140
No face detected 71140
No face detected 71141
No face detected 71141
No face dete

In [33]:
print(len(embeddings[:-17000]))
np.save('embeddings_sub10_sleepy.npy', embeddings[:-17000])

131558


In [12]:
# Extracts average embedding in a segment from given embeddigs
def embeddings_segment(embeddings, video_len, segment_len):
    # The amount of frames at the end that are not taken into account
    rest = video_len % segment_len
    num_frames = video_len - rest
    avg_embeddings = []
    acum_embeddings = np.zeros(2048)
    # a blink is counted to a segment,when the blink starts in that segment
    for frame in range(num_frames + 1):
        if frame % 1000 == 0:
            print(frame)
        acum_embeddings = acum_embeddings + np.array(embeddings[frame])
        # only happens at the end of a segment
        if frame % segment_len == 0 and frame != 0:
            avg_embeddings.append(acum_embeddings / segment_len)
            acum_embeddings = np.zeros(2048)
            #print('New segment: ', frame)
    return avg_embeddings

avg_embeddings = embeddings_segment(embeddings, len(embeddings), 100)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000


In [13]:
pca = PCA(n_components=2)
pri_comps = pca.fit_transform(avg_embeddings)

print(pri_comps)

[[ 8.30535838e+00 -7.56884424e+00]
 [ 8.38944857e+00 -8.41476329e+00]
 [ 7.54757220e+00 -1.11616900e+01]
 [ 2.17458458e+00 -6.67095071e+00]
 [ 2.75705971e+00 -9.69968774e+00]
 [ 5.83168783e+00 -6.70183016e+00]
 [ 6.06092071e+00 -8.99997825e+00]
 [ 2.78817006e+00 -6.04109889e+00]
 [ 3.99503055e+00 -5.41090215e+00]
 [ 3.29361614e+00 -5.00629792e+00]
 [ 4.29840064e+00 -4.29587538e+00]
 [ 3.66767408e+00 -3.65877341e-01]
 [ 4.75823844e+00  5.71524188e+00]
 [ 6.04284171e+00 -1.30381658e+01]
 [ 4.62330823e+00 -9.69791599e+00]
 [ 1.29094081e+00 -8.97671579e-01]
 [ 8.73677126e+00 -4.95809955e+00]
 [ 6.84223939e+00 -9.48754151e+00]
 [ 7.97005668e+00 -1.03672186e+01]
 [ 9.82511907e+00 -1.10267107e+01]
 [ 1.20430216e+01 -1.52614411e+01]
 [ 1.04652592e+01 -6.79462991e+00]
 [ 7.60956496e+00 -2.02857451e+00]
 [ 4.26947815e+00 -1.19606234e+01]
 [ 5.57418422e+00 -1.59231923e+01]
 [ 7.05382727e+00 -1.40898144e+01]
 [ 7.12071104e+00 -1.27560850e+01]
 [ 7.87357272e+00 -1.21243750e+01]
 [ 8.78499499e+00 -1